In [1]:
%pip install -U transformers datasets evaluate accelerate torch

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.9/10.1 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.1 MB 25.9 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 25.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/536.7 kB ? eta -:--:--
   --------------------------------------- 536.7/536.7 kB 17.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 33.9 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
msclap 1.3.4 requires transformers<5.0.0,>=4.34.0, but you have transformers 5.0.0 which is incompatible.
torchaudio 2.7.1 requires torch==2.7.1, but you have torch 2.10.0 which is incompatible.
torchvision 0.22.1 requires torch==2.7.1, but you have torch 2.10.0 which is incompatible.


In [2]:
import os
import json
import time
import platform
import socket
import subprocess
from datetime import datetime

import requests
import torch
from transformers import AutoTokenizer, AutoModel

c:\0-Kathy Cui\0-Northwestern\0-Classes\3rd Year\Spring 2025\CS352\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# ---------- Helpers ----------
def run(cmd):
    return subprocess.check_output(cmd, text=True).strip()

def get_imds_token(timeout=2):
    """Get IMDSv2 token (works when IMDS is enabled; required on many EC2 configs)."""
    try:
        r = requests.put(
            "http://169.254.169.254/latest/api/token",
            headers={"X-aws-ec2-metadata-token-ttl-seconds": "21600"},
            timeout=timeout,
        )
        if r.status_code == 200:
            return r.text
    except Exception:
        pass
    return None

def get_imds(path, token=None, timeout=2):
    """Query EC2 instance metadata. Returns None if not on EC2 / blocked."""
    url = f"http://169.254.169.254/latest/meta-data/{path}"
    headers = {}
    if token:
        headers["X-aws-ec2-metadata-token"] = token
    try:
        r = requests.get(url, headers=headers, timeout=timeout)
        if r.status_code == 200:
            return r.text
    except Exception:
        pass
    return None

In [4]:
token = get_imds_token()
instance_id = get_imds("instance-id", token)
instance_type = get_imds("instance-type", token)
az = get_imds("placement/availability-zone", token)

print("=== EC2 Proof ===")
print("hostname:", socket.gethostname())
print("local time:", datetime.now().isoformat(timespec="seconds"))
print("instance-id:", instance_id)
print("instance-type:", instance_type)
print("availability-zone:", az)

if instance_id is None:
    print("WARNING: Could not access EC2 metadata. You might not be on EC2, or IMDS is disabled/blocked.")
else:
    print("OK: EC2 metadata accessible (strong proof you're on an EC2 instance).")

# Optional extra evidence: show OS + kernel
print("\n=== System ===")
print("platform:", platform.platform())
try:
    print("uname -a:", run(["uname", "-a"]))
except Exception:
    pass

=== EC2 Proof ===
hostname: kathy_arz
local time: 2026-02-03T13:27:18
instance-id: None
instance-type: None
availability-zone: None

=== System ===
platform: Windows-10-10.0.26200-SP0


In [5]:
print("\n=== Python & Packages ===")
print("python:", platform.python_version())
print("torch:", torch.__version__)
print("transformers:", __import__("transformers").__version__)
print("requests:", requests.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("cuda version:", torch.version.cuda)
    print("gpu name:", torch.cuda.get_device_name(0))


=== Python & Packages ===
python: 3.11.11
torch: 2.10.0+cpu
transformers: 5.0.0
requests: 2.32.3
cuda available: False


In [6]:
out_dir = "outputs"
os.makedirs(out_dir, exist_ok=True)

payload = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "hostname": socket.gethostname(),
    "instance_id": instance_id,
    "instance_type": instance_type,
    "availability_zone": az,
    "python": platform.python_version(),
    "torch": torch.__version__,
    "transformers": __import__("transformers").__version__,
    "cuda_available": torch.cuda.is_available(),
    "checkpoint": ckpt,
    "text": text,
    "cls_embedding_shape": list(cls.shape),
    "cls_embedding_first5": cls[0, :5].tolist(),
}

path = os.path.join(out_dir, "ec2_bert_smoketest.json")
with open(path, "w", encoding="utf-8") as f:
    json.dump(payload, f, indent=2)

print("\nSaved proof file:", path)
print("Done ✅")

NameError: name 'ckpt' is not defined